### Initialization

In [1]:
import glob
import re
import textdistance
import numpy
import pandas
import editdistance
import udapi

In [2]:
from utils.text_extraction import *
from utils.document_processing import *
from utils.subject_extraction import *
from utils.similarity import *
from utils.context_extraction import *

In [3]:
import psycopg2
psycopg2_conn = psycopg2.connect(dbname='public_contracts', user='postgres', password='admin', host='localhost', port='5432')

In [4]:
from ufal.udpipe import *
# load model from the given file;
# if the file does not exist, expect a Segmentation fault
udpipe_model = Model.load("../model/udpipe/udpipe-ud-2.4-190531/czech-pdt-ud-2.4-190531.udpipe")

In [5]:
# create a UDPipe processing pipeline with the loaded model,
# with "horizontal" input (a sentence with space-separated tokens),
# default setting for tagger and parser,
# and CoNLL-U output
udp_pipeline = Pipeline(udpipe_model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu")

### Documents 

In [6]:
from utils.document_processing import *

In [7]:
loader = DatabaseDocumentLoader(psycopg2_conn)
loader.load_documents()
documents = loader.prepare_documents()

Running query: select * from document where processed=True
Preparing total 1008 documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 60.0%
Progress: 70.0%
Progress: 80.0%
Progress: 90.0%
Progress: 100.0%


In [8]:
df_documents = pandas.DataFrame(documents)

### Subject context

In [9]:
from utils.context_extraction import *

In [10]:
contract_texts = []
for contr_id, docs in df_documents.groupby('contr_id'):
    contr_text = ''
    for index, doc in docs.iterrows():
        doc_id = doc['id']
        contr_text += '======================='+str(doc_id)+'=======================\n'
        contr_text += doc['text']
    contr = {'id': contr_id, 'text': contr_text}
    contract_texts.append(contr)
df_contracts = pandas.DataFrame(contract_texts)
df_contracts

,id,text
0,2,=======================1======================...
1,3,=======================3======================...
2,6,=======================26=====================...
3,7,=======================27=====================...
4,10,=======================35=====================...
...,...,...
154,845,=======================4133===================...
155,846,=======================4134===================...
156,847,=======================4137===================...
157,848,=======================4140===================...


In [11]:
%%time

extractor = AdvancedSubjectContextExtractor()
df_contracts['subj_context'] = df_contracts['text'].apply(lambda text: extractor.process(text))

Wall time: 51.2 s


In [12]:
df_contracts

,id,text,subj_context
0,2,=======================1======================...,[Předmět smlouvy a podmínky provádění prací v ...
1,3,=======================3======================...,[Předmět smlouvy a předmět díla\n\nPředmětem t...
2,6,=======================26=====================...,[Předmět smlouvy a podmínky provádění prací v ...
3,7,=======================27=====================...,[Předmět smlouvy a předmět díla\n\nPředmětem t...
4,10,=======================35=====================...,[Předmět díla\n1. \nPředmětem smlouvy je reno...
...,...,...,...
154,845,=======================4133===================...,[Předmět veřejné zakázky\n\nPředmětem veřejné ...
155,846,=======================4134===================...,[Předmět veřejné zakázky\n\nPředmětem veřejné ...
156,847,=======================4137===================...,[Předmět smlouvy\n\nPředmětem této Smlouvy jso...
157,848,=======================4140===================...,[POPIS PŘEDMĚTU ZAKÁZKY:\n\nPředmětem zakázky ...


### Subject extraction

In [13]:
from utils.subject_extraction import *

In [14]:
from utils.subject_context_preprocessing import *

In [15]:
from utils.conllu_preprocessing import *

In [16]:
%%time

transformers = [
    NumeralLinesFilter(too_many_numerals_ratio_threshold=0.5),
    TooShortLinesFilter(too_short_line_threshold=5),
    IrrelevantLinesFilter(keywords=['strana', 'stránka', 'e-mail'], max_line_length=75, lower=True),
    IrrelevantLinesFilter(keywords=['Tel:', 'Fax:', 'IČ:', 'IČO:', 'DIČ:'], max_line_length=75, lower=False),
    IrrelevantLinesRegexFilter(patterns=[r'www', r'[\w\-\.]+\s*@\s*([\w\-]+\.)+[\w\-]{2,4}']),  # email
    IrrelevantLinesRegexFilter(patterns=[r'(\+\d{2,3}){0,1}(\s{0,1}\d{3}){3}']),  # phone
    RegexReplaceTransformer(pattern_to_transform=r',([\s]+[A-Z][a-z ])', result_pattern='.\g<1>'),
    RegexReplaceTransformer(pattern_to_transform=r'\n[ \t]*(.{0,1}[\d]+.{0,1})+[ \t]*',  # paragraph numbers
                            result_pattern='\n'),
    BlankLinesFilter(replacement='\n', top_n_frequency=200, top_n_var_threshold=5,
                     full_line_threshold=0.85, min_max_line_length=0),
    ReplaceMarksTransformer(marks_to_transform='„“', result_mark='"'),
    TooLongLinesTransformer(forbidden_delimiters='aábcčdďeéěfghiíjklmnňoópqrřsštťuúůvwxyýzž0123456789',
                            special_delimiters={'-': (r'[\s,\.](-)[\s]+[^(Kč)]', 1)},
                            too_long_line_treshold=200),
    RegexReplaceTransformer(pattern_to_transform=r'([^\n])[ ]*\n', result_pattern='\g<1>.\n'),
    RegexReplaceTransformer(pattern_to_transform=r'(([Nn]ázev|[Pp]opis)[^\n,.:"()]{5,})(\s[A-Z][^\n:]{10})',
                            result_pattern='\g<1>:\g<3>'),
    ReplaceMarksTransformer(marks_to_transform=[':'], result_mark=':.'),
    ReplaceMarksTransformer(marks_to_transform=['..'], result_mark='.'),
    RegexReplaceTransformer(pattern_to_transform=r'\([^\n()]*\)', result_pattern=''),
]
preprocessor = SubjectContextPreprocessor(transformers)
df_contracts['filtered_context'] = df_contracts['subj_context'].apply(lambda data: preprocessor.process(data))
# df_contracts

Wall time: 1.02 s


In [17]:
%%time 
attribute_transformers = [
    QuotedContractNameExtractor(),
    StructuredContractNameExtractor(),
    ItemColonExtractor(),
    StructureItemEnumerationExtractor(),
    CharItemEnumerationExtractor(),
    AttributeExtractor(keep_text=False, keep_attributes=True)
]
attribute_extractor = SubjectContextPreprocessor(transformers = attribute_transformers)
df_contracts['attributes'] = df_contracts['filtered_context'].apply(lambda data: attribute_extractor.process(data))
# df_contracts

Wall time: 675 ms


In [18]:
%%time

def tag_text(text):
    return udp_pipeline.process(text)

df_contracts['subj_context_decomposition'] = df_contracts['filtered_context'].apply(lambda data: apply_transformation(data, tag_text))
# df_contracts

Wall time: 1min 37s


In [19]:
%%time

df_contracts['subj_document'] = df_contracts['subj_context_decomposition'].apply(lambda data: apply_transformation(data, create_conllu_document))
# df_contracts

Wall time: 2.81 s


In [20]:
%%time

conllu_transformers = [
    UdapiWordOccurrencePartSentenceFilter(keywords=['cena', 'hodnota', 'DPH']),
    UdapiWordOccurrencePartSentenceFilter(keywords=['příloha', 'dále', 'jen']),
    NonSubjectPartSentenceFilter(),
    EmptyBundlesFilter(),
]
conllu_preprocessor = ConlluSubjectContextPreprocessor(transformers=conllu_transformers)
df_contracts['filtered_document'] = df_contracts['subj_document'].apply(lambda doc: conllu_preprocessor.process(doc))
# df_contracts

Wall time: 10.8 s


In [21]:
df_contracts['filtered_document_text'] = df_contracts['filtered_document'].apply(lambda data: apply_transformation(data, conllu_doc_to_str))
# df_contracts

In [23]:
tagger = AttributeTagger(attr_tag='<ITEM>;<ITEM/>', keep_text=False)

df_contracts['attributes2'] = df_contracts['filtered_document_text'].apply(lambda data: apply_transformation(data, tagger.process))
# df_contracts

In [28]:
def merge_attributes(row):
    if isinstance(row['attributes'], list):
        return [attrs + '\n' + attrs2 for attrs, attrs2 in zip(row['attributes'], row['attributes2'])]
    return row['attributes'] + '\n' + row['attributes2']

df_contracts['merged_attributes'] = df_contracts.apply(lambda row: merge_attributes(row), axis=1)
# df_contracts

### Measurements

### Playground

In [ ]:
# 10 sracky

In [60]:
index = 10
for data in df_contracts.loc[index, 'merged_attributes']:
    print(data)
    print('==========')

<ITEM>00 hod. nebo dle dohody s objednatelem. Rozsah prací a jejich četnost je vymezena v příloze č. 2 — Standardy a rozsah úklidových prací v budově Václavské náměstí 816/49. Praha 1. Výměry uklízených ploch jsou uvedeny v příloze č. 3 této smlouvy<ITEM/>
<ITEM>b)Úklid společných prostor objektů spravovaných DS<ITEM/>
<ITEM>Václavské náměstí 1282/51. Praha 1<ITEM/>
<ITEM>Nad Výšinkou 125/7. Praha 5<ITEM/>
<ITEM>Na Baště sv. Jiří 258/7. Praha 6<ITEM/>
<ITEM>Na Zátorce 339/10. Praha 6<ITEM/>
<ITEM>Standardy a četnosti prací jsou uvedeny v příloze č. 4 této smlouvy - Standardy úklidových prací pro objekty DS<ITEM/>
<ITEM>vztahy mezi smluvními stranami při zajišťování úklidových prací a zajištění dodávek čistících a hygienických prostředků v objektu Václavské náměstí 816<ITEM/>
<ITEM>00 hod. nebo dle dohody s objednatelem. Rozsah prací a jejich četnost je vymezena v příloze č. 2 – Standardy a rozsah úklidových prací v budově Václavské náměstí 816/49. Praha 1. Výměry uklízených ploch jsou 

In [61]:
for data in df_contracts.loc[index, 'filtered_context']:
    print(data)
    print('==========')

Předmět díla.

 | Předmětem této smlouvy jsou vztahy mezi smluvními stranami při zajišťování úklidových prací a zajištění dodávek čistících a hygienických prostředků v objektu Václavské náměstí 816/49. Praha 1, Václavské náměstí 1282/51. Praha 1. Nad Výšinkou 125/7. Praha 5. Na Baště sv. Jiří 258/7. Praha 6. Na Zátorce 339/10.
Praha 6.
 . Rozsah prací a dodávek je stanoven cenovou nabídkou v rozsahu položkového rozpočtu ze dne 14.10.2018, který je přílohou č. 1 této smlouvy.

| Uklidovými pracemi se rozumí:.

a) Pravidelný úklid v objektu Václavské náměstí 816/49. Praha 1.

Pravidelný úklid prováděný od pondělí do pátku od 18:.00 do 20:.00 hod. nebo dle dohody s objednatelem. Rozsah prací a jejich četnost je vymezena v příloze č. 2 — Standardy a rozsah úklidových prací v budově Václavské náměstí 816/49. Praha 1. Výměry uklízených ploch jsou uvedeny v příloze č. 3 této smlouvy.

b)Úklid společných prostor objektů spravovaných DS.

Jedná se o úklid společných prostor v objektech:.

Václa

In [62]:
for data in df_contracts.loc[index, 'subj_context']:
    print(data)
    print('==========')

Předmět díla

1. | Předmětem této smlouvy jsou vztahy mezi smluvními stranami při zajišťování
úklidových prací a zajištění dodávek čistících a hygienických prostředků v objektu
Václavské náměstí 816/49, Praha 1, Václavské náměstí 1282/51, Praha 1, Nad
Výšinkou 125/7, Praha 5, Na Baště sv. Jiří 258/7, Praha 6, Na Zátorce 339/10,
Praha 6.

2.. Rozsah prací a dodávek je stanoven cenovou nabídkou v rozsahu položkového
rozpočtu ze dne 14.10.2018, který je přílohou č. 1 této smlouvy.

3. | Uklidovými pracemi se rozumí:

a) Pravidelný úklid v objektu Václavské náměstí 816/49, Praha 1

Pravidelný úklid prováděný od pondělí do pátku od 18:00 do 20:00 hod. nebo dle
dohody s objednatelem. Rozsah prací a jejich četnost je vymezena v příloze č. 2 —
Standardy a rozsah úklidových prací v budově Václavské náměstí 816/49, Praha 1.
Výměry uklízených ploch jsou uvedeny v příloze č. 3 této smlouvy.

b)Úklid společných prostor objektů spravovaných DS

Jedná se o úklid společných prostor v objektech:

Václa

In [40]:
text = df_contracts.loc[index, 'text']
print(text)

=======================3766=======================
General description:

The EVC2 visa information system (supplier: IBM Česká republika, spol. s r.o.) uses 4-4-2
biometric scanners accessed via heavy clients on the C, C++, Cf and Microsoft NET

Framework (at least 2.0. version) basis. Fingerprint images are processed in EVC2 clients.

The NS-VIS information system (supplier: IBM Česká republika, spol. s r.o.) uses 4-4-2 and
1-finger biometric scanners accessed via heavy clients on the Microsoft .NET Framework

basis (at least 2.0. version), communicating with the application server via web services.
4-4-2 scanners must conform to the following specifications:
e flat and rolled fingerprint capture

e | single and dual fingerprint capture (two thumbs or other fingers) and four-fingerprint

capture (1, 2 and 4 fingers)
e | automatic capture without operator intervention
e filtering and rejecting residual ghost fingerprint images
e | liveness detection (against fingerprint stamps and othe